<a href="https://colab.research.google.com/github/ziqlu0722/Machine-Learning/blob/master/Generative-Algorithm/GDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Algorithms

Gerative Models are density estimation.

## Ultimate Task: Estimate $P(y\mid x)$

This Task of generative models is equivelent to: 
##Estimate $P(x \mid y)$

According to Bayes Rule: $$ P(x^1, x^2, x^3..., x^d \mid y) = \frac{P(y \mid x^1, x^2, x^3..., x^d ) \, P(y)}{P(x^1, x^2, x^3..., x^d )} $$

*1.$ P(x^1, x^2, x^3..., x^d \mid y)$  is the same for all y, not a factor in ranking $P(y\mid x)$*

*2.$ P(y)$ is prior, can be calculated from training counts if label is provided*

___

Below descriptions from [source](https://towardsdatascience.com/gaussian-discriminant-analysis-an-example-of-generative-learning-algorithms-2e336ba7aa5c) talkes about the difference between discriminative and generative algorithms

## 1. Discriminative Learning Algorithms: 

In Linear Regression and Logistic Regression both we modelled conditional distribution of $y$ given $x$. Algorithms that model $P(x \mid y)$ directly from the training set are called discriminative algorithms. 

## 2. Generative Learning Algorithms: 

There can be a different approach to the same problem, consider the same binary classification problem where we want learn to distinguish between two classes, class $A(y=1)$ and class $B (y=0)$ based on some features. Now we take all the examples of label $A$ and try to learn the features and build a model for class $A$. Then we take all the examples labeled $B$ and try to learn it’s features and build a separate model for class $B$. Finally to classify a new element, we match it against each model and see which one fits better (generate high value for probability). In this approach we try to model $P(x\mid y)$  and $P(y)$ as oppose to $P(y)$ we did earlier, it’s called Generative Learning Algorithms. Once we learn the model $P(y)$ and $P(x\mid y)$  using training set, we use Bayes Rule to derive the $P(y\mid x)$. 

All in all, 
* generative approch >> model $P(x\mid y)$ and $P(y)$ to estimate $P(y\mid x)$ indirectly based on Bayes Rule
* discriminative algorithm >> model $P(x\mid y)$ directly

## How to Calculate $P(x^1, x^2, ..., x^d\mid y)?$

- Option1: Assume Feature Independence - **Naive Bayes**

- Option2: Model (restrict the joint distribution) - **GDA(gaussian)/EM(mixture)**
  
  - typically with a well known parameterized form
  - estimate the parameters of the imposed model

- Option3: Mix, Bend, Tweak Option 1 and 2 - **Bayesian Network/Graphical Model**
  
  - don't fully factorize by independence like Naive Bayes
  - e.g. $P(x^1\mid y) * P(x^2, x^3\mid y) * P(x^4\mid y)...$




#Practice GDA

### Problem

Spam or Non-Spam ? - Binary Classification Problem

###Step1: Load the Data

In [0]:
import numpy as np
import collections

In [51]:
data = []

with open('drive/Machine-Learning/data/spambase/spambase.txt') as file:
  for line in file:
    line_split = line.strip().split(',')
    data.append([float(c) for c in line_split])
#     data.append(line.strip().split(','))

print('There are {} data instances'. format(len(data))) 
print(collections.Counter(np.array(data)[:,-1]))

There are 4601 data instances
Counter({0.0: 2788, 1.0: 1813})


### Step 2: K-Folds Cross Validation Help Function

In K-Folds Cross Validation we split our data into k different subsets (or folds). We use k-1 subsets to train our data and leave the last subset (or the last fold) as test data. We then average the model against each of the folds and then finalize our model. After that we test it against the test set. (cited from [source](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6))

![](https://cdn-images-1.medium.com/max/1600/1*J2B_bcbd1-s1kpWOu_FZrg.png)
*Thanks to [Joseph Nelson](https://medium.com/@josephofiowa)  for this visual*

In [0]:
from random import randrange

# Split a dataset into k folds
def cross_validation_split(dataset, folds=5):

  dataset_split = []     
  dataset_copy = list(dataset)
  fold_size = int(len(dataset) / folds)
  for i in range(folds):
    fold = []
    while len(fold) < fold_size:
      index = randrange(len(dataset_copy))
      fold.append(dataset_copy.pop(index))
    dataset_split.append(fold)

  idx = 0
  test_set = []
  train_set = []
  for i in range(folds):
    test_set.append(dataset_split[idx])
    train_set_j = []
    train_set_k = []
    for j in dataset_split[:idx] + dataset_split[idx + 1:]:
      for k in j:
        train_set_k.append(k)
    train_set.append(train_set_k)
    idx += 1
  
  return train_set, test_set

###Step 3. GDA Algorithm Implementation

**1. Maximum Likelihood**

For supervised classification, we could model class conditional distribution of the data $P(X|y=k)$ for each class. Predictions can then be obtained by using Bayes’ rule: 
$$P(y=k | X) = \frac{P(X | y=k) P(y=k)}{P(X)} = \frac{P(X | y=k) P(y = k)}{ \sum_{l} P(X | y=l) \cdot P(y=l)}$$

More specifically, for given data, we predict $P(y=k | X)$ by calculate, and compare the conditional probability in each class (right of the equation above), and then select the class which maximizes this conditional probability.

$P(X | y=k)$ is derived from **Multivariate Gaussian distribution.**

Multivariate Gaussian distribution, is parameterized by a mean vector $µ ∈ R^d$
and a covariance matrix $Σ ∈ R^{d*d}$
, where $Σ ≥ 0$ is symmetric and positive
semi-definite. Also written $N (µ, Σ)$, its density is given by:

$$P(X | y=k) = \frac{1}{(2\pi)^{d/2} |\Sigma_k|^{1/2}}\exp\left(-\frac{1}{2} (X-\mu_k)^t \Sigma_k^{-1} (X-\mu_k)\right)$$


where $d$ is the number of features, 
$|\Sigma_k|$ is the determinant of $\Sigma$


We can estimate from training data that:

- $P(y=k)$ is: the proportion of instances of class

- $\mu_k$: empirical sample class means

- $ \Sigma_k$: either by the empirical sample class covariance matrices, or by a regularized estimator

---
In the case of LDA, the Gaussians for each class are assumed to share the same covariance matrix:  for all . This leads to linear decision surfaces, which can be seen by comparing the log-probability ratios: $\log[P(y=k | X) / P(y=l | X)$]

In the case of QDA, there are no assumptions on the covariance matrices  of the Gaussians, leading to quadratic decision surfaces.

---
*reference:* *[Andrew Ng](http://cs229.stanford.edu/notes/cs229-notes2.pdf)*, *[Scikit-learn Document](https://scikit-learn.org/stable/modules/lda_qda.html)*

**2. Calculate Covariance Matrix:**

$ X = 
  \begin{bmatrix}
    x_{11} &...x_{1j}...& x_{1d}  \\
    ... & ... & ...\\
    x_{i1} &...x_{ij}...& x_{id}  \\
    ... & ... & ...\\
    x_{n1}  &...x_{nj}...& x_{nd}  \\
  \end{bmatrix}$

*Where $j\in (0,d), i\in (0, n)$*

*$X: n \times d$ matrix*

$Cov(X)= N^{-1}X^TX 
= N^{-1} \times \begin{bmatrix}
    x_{11} &...x_{i1}...& x_{n1}  \\
    ... & ... & ...\\
    x_{1j} & ...x_{ij}...& x_{ni}  \\
    ... & ... & ...\\
    x_{1d} &...x_{id}...&x_{nd}  \\
  \end{bmatrix} \times 
  \begin{bmatrix}
    x_{11} &...x_{1j}...& x_{1d}  \\
    ... & ... & ...\\
    x_{i1} &...x_{ij}...& x_{id}  \\
    ... & ... & ...\\
    x_{n1}  &...x_{nj}...& x_{nd}  \\
  \end{bmatrix} 
=  N^{-1} \times
  \begin{bmatrix}
    \sum_{i=1}^nΣ x_{i1}^2  &  ...\sum_{i=1}^n x_{i1}x_{ij}. . .    &	\sum_{i=1}^n x_{i1}x_{id}	 \\
        ... & ... & ...\\
    \sum_{i=1}^n x_{i1}x_{ij}  &  	...\sum_{i=1}^nΣ x_{ij}^2. . .    &	\sum_{i=1}^nx_2x_d \\
        ... & ... & ...\\
    \sum_{i=1}^n x_{i1}x_{id}	  &  	...\sum_{i=1}^nΣ x_{id}x_{ij}	. . .    &	\sum_{i=1}^nx_{id}^2 \\
  \end{bmatrix}
=  N^{-1} \times
  \begin{bmatrix}
    var(x_1)  &  ...cov(x_1,x_j). . .    &	cov(x_1, x_d)	 \\
        ... & ... & ...\\
    cov(x_1, x_j)  &  	...var(x_{ij}). . .    &	cov(x_2, x_d) \\
        ... & ... & ...\\
    cov(x_1, x_d)	  &  	...cov(x_d, x_j)	. . .    &	var(x_d) \\
  \end{bmatrix}$
  
  *$Cov(X): d \times d$ matrix*


In [0]:
class Metric:
  def __init__(self, predict, label):
    self.predict = predict
    self.label = label
    self.num_obs = len(label)
  
  def acc(self):
    err = 0
    for i in range(len(self.label)):
      if self.predict[i] != self.label[i]:
          err += 1
    acc = err/len(self.label)
    print('Accuracy---{}'.format(acc))
    return acc
  
#   def roc(self):
#     true_positives = 0
#     true_negatives = 0
#     false_positives = 0
#     false_negatives = 0

#     for i in range(len(self.label)):
#       if self.predict[i] == self.label[i] == 1:
#           true_positives += 1
#       elif self.predict[i] == self.label[i] == 0:
#           true_negatives += 1
#       elif self.predict[i] == 1 and self.label[i] == 0:
#           false_positives += 1
#       elif self.predict[i] == 0 and self.label[i] == 1:
#           false_negatives += 1

#     tpr =  (true_positives) / (true_positives + false_negatives)
#     fpr = (false_positives) / (true_negatives + false_positives)
#     return tpr, fpr

In [0]:
from scipy.stats import multivariate_normal

class GDA:
  
  def __init__(self, data):
    """split the data set by classes"""
    # split data into x and y
    x = np.array(data)[:,:-1]
    y = np.array(data)[:,-1]
    
    # split data for each class:
    self.input = {'c0': x[y==0], 'c1':x[y==1]}

    # calculate the prior as the frequency of class over all data points
    self.prior = {'c0': 1 - y.mean(), 'c1': y.mean()}
    
    # calculate the mean for each x set:
    self.mu = {'c0': x[y==0].mean(axis = 0), 'c1': x[y==1].mean(axis = 0)}
    centered_x0 = np.mat(self.input['c0'] - self.mu['c0'])
    centered_x1 = np.mat(self.input['c1'] - self.mu['c1'])
    centered_x = np.mat(np.concatenate([centered_x0, centered_x1]))
    
    # calculate the sigma for each x set:
    self.sigma = (1.0/len(y))*(centered_x.T @ centered_x)
  
  def predict(self, test_data):
    test_x = np.array(test_data)[:,:-1]
    predict = []
    c0_pdf = multivariate_normal.pdf(test_x, self.mu['c0'], self.sigma)
    c1_pdf = multivariate_normal.pdf(test_x, self.mu['c1'], self.sigma)
    for i in range(len(test_data)):
      if self.prior['c0'] * c0_pdf[i] < self.prior['c1'] * c1_pdf[i]:
        predict.append(0)
      else:
        predict.append(1)
    return predict
  
  def evaluate(self, test_data):
    test_x = np.array(test_data)[:,:-1]
    label = np.array(test_data)[:,-1]
    predict = self.predict(test_data)
    metric = Metric(predict, label)
    return metric

**Step 4: Run the Algorithm with Cross-Validation **

In [204]:
# set hyper parameters
num_folds = 10

train_set, test_set = cross_validation_split(data, folds = num_folds)

for i in range(num_folds):
  acc = GDA(train_set[i]).evaluate(test_set[i]).acc()
print('The Accuracy Over 10 folds Cross Validation is {}'.format(np.mean(acc)))

Accuracy---0.9
Accuracy---0.8717391304347826
Accuracy---0.8956521739130435
Accuracy---0.9108695652173913
Accuracy---0.8913043478260869
Accuracy---0.8673913043478261
Accuracy---0.8673913043478261
Accuracy---0.9021739130434783
Accuracy---0.8934782608695652
Accuracy---0.8695652173913043
The Accuracy Over 10 folds Cross Validation is 0.8695652173913043
